In [1]:
import pandas as pd

In [2]:
url = '../mail.csv'
dtype={'To':'string', 'Subject':'string', 'X-From':'string',
       'X-To':'string', 'X-cc':'string', 'X-bcc':'string', 'X-Folder':'string', 'X-Origin':'string', 'X-FileName':'string',
       'content':'string', 'user':'string', 'Cat_1_level_1':'string', 'Cat_1_level_2':'string', 'Cat_1_weight':'string',
       'Cat_2_level_1':'string', 'Cat_2_level_2':'string', 'Cat_2_weight':'string', 'Cat_3_level_1':'string',
       'Cat_3_level_2':'string', 'Cat_3_weight':'string', 'Cat_4_level_1':'string', 'Cat_4_level_2':'string',
       'Cat_4_weight':'string', 'Cat_5_level_1':'string', 'Cat_5_level_2':'string', 'Cat_5_weight':'string',
       'Cat_6_level_1':'string', 'Cat_6_level_2':'string', 'Cat_6_weight':'string', 'Cat_7_level_1':'string',
       'Cat_7_level_2':'string', 'Cat_7_weight':'string', 'Cat_8_level_1':'string', 'Cat_8_level_2':'string',
       'Cat_8_weight':'string', 'Cat_9_level_1':'string', 'Cat_9_level_2':'string', 'Cat_9_weight':'string',
       'Cat_10_level_1':'string', 'Cat_10_level_2':'string', 'Cat_10_weight':'string', 'Cat_11_level_1':'string',
       'Cat_11_level_2':'string', 'Cat_11_weight':'string', 'Cat_12_level_1':'string', 'Cat_12_level_2':'string',
       'Cat_12_weight':'string'}
data = pd.read_csv(url,dtype=dtype)

In [3]:
import string 

data['content'][2]
sum([i.strip(string.punctuation).isalpha() for i in data['content'][2].split()]) 

5

In [4]:
data['content_char_len'] = data['content'].apply(lambda x: len(x)  if (type(x)==type('str')) else 0)

In [5]:
#data.boxplot(column=['content_len'])
#Boxplot inexploitable

In [6]:
data['content_char_len'].describe()

count    100000.00000
mean       2158.93104
std        7370.24427
min           0.00000
25%         272.00000
50%         758.00000
75%        1854.00000
max      260461.00000
Name: content_char_len, dtype: float64

In [13]:
data['content_char_len'].quantile(0.90)

content_char_length_q1 = data['content_char_len'].quantile(0.25)
content_char_length_q2 = data['content_char_len'].quantile(0.75)


In [9]:
data['content_char_length_category'] = data['content_char_len'].apply(lambda x: 'short message' if x<content_char_length_q1 else ('average message' if x<content_char_length_q2 else 'long message'))

In [10]:
data['content_char_length_category']

0          short message
1        average message
2          short message
3          short message
4          short message
              ...       
99995    average message
99996    average message
99997    average message
99998       long message
99999    average message
Name: content_char_length_category, Length: 100000, dtype: object

In [15]:
data['content_word_len'] = data['content'].apply(lambda x: sum([i.strip(string.punctuation).isalpha() for i in x.split()])  if (type(x)==type('str')) else 0)

In [16]:
data['content_word_len'].describe()

count    100000.00000
mean        296.20993
std        1034.19512
min           0.00000
25%          38.00000
50%         105.00000
75%         253.00000
max       38084.00000
Name: content_word_len, dtype: float64

In [21]:
print(data['content_word_len'].quantile(0.95))

content_word_length_q1 = data['content_word_len'].quantile(0.25)
content_word_length_q2 = data['content_word_len'].quantile(0.75)


929.0


In [18]:
data['content_word_length_category'] = data['content_word_len'].apply(lambda x: 'short message' if x<content_word_length_q1 else ('average message' if x<content_word_length_q2 else 'long message'))